In [ ]:
from time import sleep
import numpy as np
from os.path import dirname, join, isfile, isdir
from os import makedirs
from sys import exit
import pandas as pd
from Levenshtein import ratio
from datetime import datetime, timedelta

from bulletin.utils import static, normalize
from bulletin.sivep import sivep
from bulletin.sim import sim
from bulletin.notifica import Notifica
from bulletin.utils.utils import Timer, get_better_notifica
from bulletin.utils.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

In [ ]:
#notifica = Notifica()
#notifica.load()
#notifica.shape()

In [ ]:
sivep = sivep()
sivep.read_all_database_files()
sivep.to_notifica()
sivep.shape()

In [ ]:
sim = sim()
sim.read_all_database_files()
sim.to_notifica()
sim.shape()

In [ ]:
sim_obitos = sim.get_obitos()
sivep_obitos = sivep.get_obitos()

In [ ]:
sivep_obitos = sivep_obitos[['numero_do', 'data_cura_obito', 'paciente', 'nome_mae', 'data_nascimento', 'idade', 'sexo',
                    'raca_cor', 'cod_cbo', 'uf_residencia', 'ibge_residencia', 'bairro_residencia', 'logradouro_residencia',
                    'cep_residencia', 'uf_unidade_notifica', 'ibge_unidade_notifica', 'evolucao', 'sistema', 'id', 'cns']]

In [ ]:
obitos = sivep_obitos.append(sim_obitos, ignore_index = True)

In [ ]:
crossmatch = pd.DataFrame()
crossmatch = obitos.apply(lambda x: obitos.apply(lambda y: ratio(x['paciente'], y['paciente']), axis=1), axis=1)
crossmatch

In [ ]:
#EXECUTAR DAQUI PRA BAIXO PORQUE ESSE CROSSMATCH DEMORA UMA VIDA
for x in obitos.index:
    obitos.loc[crossmatch[x] > 0.9, 'levenshtein'] = x

In [ ]:
obitos_match_lv = obitos.loc[obitos.duplicated(['levenshtein', 'data_nascimento', 'data_cura_obito'], keep = False)]
obitos_match_lv.shape

In [ ]:
obitos_match_nm = obitos.loc[obitos.duplicated(['paciente', 'data_nascimento', 'data_cura_obito'], keep = False)]
obitos_match_nm.shape

In [ ]:
sim_obitos_lv = obitos_match_lv.loc[obitos_match_lv['sistema'] == 'SIM']
sim_obitos_lv = sim_obitos_lv.drop_duplicates(['levenshtein', 'data_nascimento', 'data_cura_obito'], keep = 'last')
sim_obitos_lv.shape

In [ ]:
sim_obitos_nm = obitos_match_nm.loc[obitos_match_nm['sistema'] == 'SIM']
sim_obitos_nm = sim_obitos_nm.drop_duplicates(['paciente', 'data_nascimento', 'data_cura_obito'], keep = 'last')
sim_obitos_nm.shape

In [ ]:
sivep_obitos_lv = obitos_match_lv.loc[obitos_match_lv['sistema'] == 'SIVEP']
sivep_obitos_lv = sivep_obitos_lv.drop_duplicates(['levenshtein', 'data_nascimento', 'data_cura_obito'], keep = 'last')
sivep_obitos_lv = sivep_obitos_lv.rename(columns={'id': 'id_sivep'})
sivep_obitos_lv = sivep_obitos_lv[['levenshtein', 'id_sivep']]
sivep_obitos_lv.shape

In [ ]:
sivep_obitos_nm = obitos_match_nm.loc[obitos_match_nm['sistema'] == 'SIVEP']
sivep_obitos_nm = sivep_obitos_nm.drop_duplicates(['paciente', 'data_nascimento', 'data_cura_obito'], keep = 'last')
sivep_obitos_nm = sivep_obitos_nm.rename(columns={'id': 'id_sivep'})
sivep_obitos_nm = sivep_obitos_nm[['paciente', 'id_sivep']]
sivep_obitos_nm.shape

In [ ]:
#VERIFIQUE ANTES O SHAPE DO DF SIM E SIVEP ANTES DE EXECUTAR ISSO
merge_lv = pd.merge(left=sim_obitos_lv, right=sivep_obitos_lv, how='left', on='levenshtein')
merge_lv.shape

In [ ]:
merge_nm = pd.merge(left=sim_obitos_nm, right=sivep_obitos_nm, how='left', on='paciente')
merge_nm.shape

In [ ]:
with pd.ExcelWriter('merge_levenshtein.xlsx') as writer:  
    merge_lv.to_excel(writer)

In [ ]:
merge_nm = merge_nm.drop_duplicates('numero_do')
merge_nm.shape

In [ ]:
merge_lv = merge_lv.drop_duplicates('numero_do')
merge_lv.shape

In [ ]:
with pd.ExcelWriter('merge_nome.xlsx') as writer:  
    merge_nm.to_excel(writer)

In [ ]:
sivep_obitos = sivep.get_obitos()
sivep_obitos.shape

In [ ]:
sivep_obitos = sivep_obitos[['id', 'data_notificacao', 'data_1o_sintomas', 'cpf', 'gestante', 'periodo_gestacao', 'etnia',
                             'febre', 'tosse', 'dor_garganta', 'dispneia', 'dispneia',
                             'saturacao_o2', 'diarreia', 'nausea_vomitos', 'outros_sintomas', 'puerperio', 'doenca_cardiovascular',
                             'sindrome_down', 'doenca_hepatica', 'diabetes', 'doenca_neurologica', 'doenca_pulmonar', 'imunodeficiencia',
                             'doenca_renal', 'obesidade', 'outras_morbidades', 'uso_antiviral', 'hospitalizado', 'tipo_internacao',
                             'raiox_torax', 'coleta_amostra', 'data_coleta', 'requisicao', 'data_liberacao', 'resultado', 'lab_executor',
                             'classificacao_final', 'criterio_classificacao', 'historico_viagem', 'local_viagem', 'data_ida_local', 'data_retorno_local',
                             'perda_olfato_paladar', 'tomografia']]
sivep_obitos = sivep_obitos.rename(columns={'id': 'id_sivep'})

In [ ]:
#TEM QUE TESTAR ISSO AI *deve mostrar um dataframe
#cm = sivep_obitos['id_sivep'].apply(lambda x: merge_lv['id_sivep']==x)
#sivep_obitos.loc[cm[1]]
#sivep_obitos.shape

In [ ]:
#merge = pd.merge(left=merge, right=sivep_obitos, how='left', on='id_sivep')

In [ ]:
#with pd.ExcelWriter('merge_sim_sivep.xlsx') as writer:  
#    merge.to_excel(writer)